# Bike Sharing Dataset Exogenous Variables Selection 

In this notebook, you will be guided on how to leverage the exogenous variables we provide in order to improve the performance of the bike-sharing dataset. The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).

Copy the code to the current working example

In [1]:
!cp ../calculate_feature_score.py .
!cp ../feature_selection.py .
!cp ../utils.py .

## Standard imports


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os, warnings, requests, sys

# Ignore warnings
warnings.filterwarnings(action='ignore')

## Download the bulk dataset from the API

Enter your API key to get access to the API

In [3]:
import getpass
api_key = getpass.getpass('Enter your Notional API key: ')

Enter your Notional API key: ········


In [4]:
url = "https://api.notional.ai/v1/series/bulk"
headers = {
  "x-notionalai-api-key": api_key,
}

response = requests.request("GET", url, headers=headers)

In [5]:
import urllib.request
import sys


opener = urllib.request.build_opener()
urllib.request.install_opener(opener)
urllib.request.urlretrieve(response.json()['result_url'], './data/all_features.parquet');

## Data preparation

Define the loss function that you want. Here we will use root mean squared error.

In [2]:
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

Your dataset should be in tabular format with a `date/timestamp column` and a `target column`. All other columns will be considered as exogenous variables.

In [4]:
# Choose your lost function here
scoring = rmse

# Date/timestamp column
timestamp_col = 'date'

# Target column, i.e label
target_col = 'count'

# Path to the bulk dataset parquet file
features_parquet_path = 'data/all_features.parquet'

# The directory to store the feature evaluation results
output_dir = 'fs_results'

# Number of trials for optuna hyperparameter tuning
optuna_n_trials = 200

# Forecast length
prediction_length = 14

We'll read and split the dataset into a train set and a test set. In addition, we will also get the `cvs` variable, which contain the validation split for our cross validation process.

In [5]:
from utils import prepare_train_val_test_data

# Read the bike sharing dataset
data = pd.read_csv("data/bike_sharing_day.csv")

# Currenly we support the timestamp column as string type
data[timestamp_col] = data[timestamp_col].astype(str)
data = data.sort_values(timestamp_col).reset_index(drop=True)

# The length of the test dataset. Set it to None for it to equals to the prediction_length
test_size = None

# The ratio of the validation dataset used for cross validation
val_ratio = 0.25

# Number of cross validation folds
cv_fold = 5

# Should we add a lag_<prediction_length> column to the dataset? Should be yes in most of the cases.
add_lag_col = True

train_data, test_data, cvs = prepare_train_val_test_data(
    data=data, 
    target_col=target_col, 
    timestamp_col=timestamp_col, 
    test_size=test_size, 
    val_ratio=val_ratio, 
    cv_fold=cv_fold, 
    prediction_length=prediction_length, 
    add_lag_col=add_lag_col
)

In [8]:
train_data['count'] = np.log1p(train_data['count'])
test_data['count'] = np.log1p(test_data['count'])
train_data['count_lag_14'] = np.log1p(train_data['count_lag_14'])
test_data['count_lag_14'] = np.log1p(test_data['count_lag_14'])

## Feature Selection

Our feature selection method consists of multiple steps to ensure significant improvement and the applicability of selected features to a wide range of time series forecasting models, even though the method is built solely on the XGBoost model. To utilize our feature selection method, follow these steps:

1. Create an instance of the FeatureSelector class.
2. Call the `fit` method on the created instance, providing the necessary parameters. Note that this process requires a machine with a GPU.
3. Once the feature selection process is completed, you can use the get_best_features() method to obtain a list of features with strong predictive power.

In [7]:
from feature_selection import FeatureSelector
feature_selector = FeatureSelector()

In [ ]:
feature_selector.fit(
    train_data=train_data,
    cvs=cvs,
    timestamp_col=timestamp_col,
    target_col=target_col,
    prediction_length=prediction_length,
    features_parquet_path=features_parquet_path,
    output_dir=output_dir,
    scoring=scoring,
    optuna_n_trials=optuna_n_trials,
    gpu_id=0,
    fitted=False #Refit?
)

Once the `feature_selector.fit()` method is invoked, it generates a directory named according to the `output_dir` variable to hold the evaluation results. In the future, if you wish to utilize the stored results without retraining, you can simply employ `feature_selector.fit()` with the fitted=True parameter.

Get top 5 features subset

In [9]:
selected_features = feature_selector.get_n_best_features(5)
selected_features

[['WCR_PRCP_00000267', 'WCR_PRCP_00000263'],
 ['WCR_PRCP_00002702', 'WCR_PRCP_00000867', 'WCR_PRCP_00000270'],
 ['WCR_PRCP_00000267'],
 ['WSR_PRCP_00000051'],
 ['WCR_PRCP_00000270']]

# Evaluate performance of selected features on different models

Import necessary modules and helper functions

In [11]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from utils import ARIMAModel
import xgboost as xgb

from utils import fine_tune_model, evaluate_models, add_exo_features

We will assess the performance of our selected features using four different time series forecasting models: SARIMAX, Lasso, XGBoost, and RandomForest. This evaluation aims to determine the robustness of the selected features.

We will evaluate the model performance with and without the selected features and make comparisons. To ensure the reliability of the evaluation, it will be conducted on unseen data, guaranteeing the significance and generalizability of the results.

In [16]:
fine_tune_model_args = {
    'train_data': train_data, 
    'target_col': target_col, 
    'cvs': cvs, 
    'scoring': scoring, 
    'timestamp_col': timestamp_col, 
    'optuna_n_trials': optuna_n_trials
}

arima_model = ARIMAModel()
lasso_model = fine_tune_model('lasso', **fine_tune_model_args)
xgb_model = fine_tune_model('xgboost', **fine_tune_model_args)
# rf_model = fine_tune_model('random_forest', **fine_tune_model_args)

models = [arima_model, lasso_model, xgb_model]
evaluate_models(models, train_data, test_data, target_col, timestamp_col, scoring, prediction_length)

Model name: ARIMAModel
Loss: 1485.3088843066928
Model name: Lasso
Loss: 1809.4064113960876
Model name: XGBRegressor
Loss: 1517.2778732478303
Best model: ARIMAModel
Best loss: 1485.3088843066928


In [17]:
train_data_final = add_exo_features(
    train_data, 
    timestamp_col, 
    selected_features[0], 
    features_parquet_path, 
    prediction_length
)

test_data_final = add_exo_features(
    test_data, 
    timestamp_col, 
    selected_features[0], 
    features_parquet_path, 
    prediction_length
)

In [18]:
fine_tune_model_args = {
    'train_data': train_data_final, 
    'target_col': target_col, 
    'cvs': cvs, 
    'scoring': scoring, 
    'timestamp_col': timestamp_col, 
    'optuna_n_trials': optuna_n_trials
}

arima_model = ARIMAModel()
lasso_model = fine_tune_model('lasso', **fine_tune_model_args)
xgb_model = fine_tune_model('xgboost', **fine_tune_model_args)
# rf_model = fine_tune_model('random_forest', **fine_tune_model_args)

models = [arima_model, lasso_model, xgb_model]
evaluate_models(models, train_data_final, test_data_final, target_col, timestamp_col, scoring, prediction_length)

Model name: ARIMAModel
Loss: 1638.281730928382
Model name: Lasso
Loss: 1963.0720095027534
Model name: XGBRegressor
Loss: 1295.9473076156137
Best model: XGBRegressor
Best loss: 1295.9473076156137
